In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
movies=pd.read_csv('D:\\recommendation system task\\movies_metadata.csv')

C:\Users\abishek\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies.tail(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [4]:
df=movies[['overview','title']]

In [5]:
###cleaning the text
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abishek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import re

In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    ##text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [8]:
df['overview'].apply(lambda x: len(x.split(' '))).sum()

AttributeError: 'float' object has no attribute 'split'

#The above error is due to the column overview has mixed datatype So,inorder to preprocess we need to change to String

In [9]:
df['overview'].dtypes

dtype('O')

In [10]:
df.dtypes

overview    object
title       object
dtype: object

overview column has mixed data dtypes so converting into string

In [11]:
df['overview']=df['overview'].astype(str)

C:\Users\abishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Before cleaning the overview column is has around 24 lakhs words

In [12]:
df['overview'].apply(lambda x: len(x.split(' '))).sum()

2456065

In [13]:
df['overview'] = df['overview'].apply(clean_text)

C:\Users\abishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Now you can see that word count has decreased to 14 lakhs.
This is due to removal of stopwords and noise.

In [14]:
df['overview'].apply(lambda x: len(x.split(' '))).sum()

1404536

In [15]:
df.head(5)

,overview,title
0,led woody andys toys live happily room andys b...,Toy Story
1,siblings judy peter discover enchanted board g...,Jumanji
2,family wedding reignites ancient feud nextdoor...,Grumpier Old Men
3,cheated mistreated stepped women holding breat...,Waiting to Exhale
4,george banks recovered daughters wedding recei...,Father of the Bride Part II


In [16]:
df.shape

(45466, 2)

In [23]:
plot=df.iloc[0,0]

In [24]:
plot

'led woody andys toys live happily room andys birthday brings buzz lightyear onto scene afraid losing place andys heart woody plots buzz circumstances separate buzz woody owner duo eventually learns put aside differences'

In [27]:
df.set_index('title', inplace = True)
df.head()

,overview
title,
Toy Story,led woody andys toys live happily room andys b...
Jumanji,siblings judy peter discover enchanted board g...
Grumpier Old Men,family wedding reignites ancient feud nextdoor...
Waiting to Exhale,cheated mistreated stepped women holding breat...
Father of the Bride Part II,george banks recovered daughters wedding recei...


In [129]:
df.columns

Index(['overview'], dtype='object')

My system could not able to all 45 thousands rows for calculating cosine similarity so i just took sample of 30000 rows

In [87]:
df_sample=df.iloc[0:30000,]

In [88]:
df_sample.tail(5)

,overview
title,
Seventeen Again,cat gene divorced several years visiting son g...
Sweet Sixteen,beautiful lonely girl named melissa tries make...
The Disappearance of Garcia Lorca,journalist starts investigation disappearance ...
The Dramatics: A Comedy,work actress unexpectedly lands starring role ...
Up the Creek,bumbling navy officer lieutenant humphrey fair...


# Feature extraction Bag of words
Converting words to vectors



In [89]:

count = CountVectorizer()
count_matrix = count.fit_transform(df_sample['overview'])

In [90]:
count_matrix.toarray().shape

(30000, 69001)

In [91]:
indices = pd.Series(df_sample.index)
indices[:5]

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

# generating the cosine similarity matrix

In [92]:

cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.02087414, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02087414, 1.        , 0.02450715, ..., 0.        , 0.        ,
        0.04494666],
       [0.        , 0.02450715, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.03656362],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.04494666, 0.        , ..., 0.03656362, 0.        ,
        1.        ]])

In [99]:
# function that takes in movie title as input and returns the top 20 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 20 most similar movies
    top_20_indexes = list(score_series.iloc[1:21].index)
    
    # populating the list with the titles of the best 20 matching movies
    for i in top_20_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [100]:
recommendations('Toy Story')

['Toy Story 2',
 'Toy Story 3',
 'Small Fry',
 'Condorman',
 'Group Sex',
 'Rebel Without a Cause',
 'Malice',
 'Tension at Table Rock',
 'For Your Consideration',
 'The Sunchaser',
 'Paris-Manhattan',
 'Halloweentown',
 'Raw Deal: A Question Of Consent',
 'Listen, Darling',
 'Hawaiian Vacation',
 'Indecent Proposal',
 'The Wild',
 'Frankenstein Conquers the World',
 'Africa Screams',
 'Window to Paris']

In [101]:
recommendations('Jumanji')

['Table No. 21',
 'Brainscan',
 'Word Wars',
 'Minecraft: The Story of Mojang',
 'Stay Alive',
 'How to Make a Monster',
 'Quintet',
 'The Last Starfighter',
 'Masques',
 'The Beast Must Die',
 'eXistenZ',
 'Nirvana',
 'DeVour',
 'Paintball',
 'Chasing Ghosts: Beyond the Arcade',
 'The Last of Sheila',
 'The Odds',
 'Panic Room',
 'Dungeons & Dragons',
 'Poolhall Junkies']

In [55]:
df_sample.loc['Peter Pan'].at['overview']

'leaving safety nursery behind wendy michael john follow peter pan magical world childhood lasts forever neverland kids must face captain hook foil attempts get rid peter good'

In [56]:
df_sample.loc['Jumanji'].at['overview']

'siblings judy peter discover enchanted board game opens door magical world unwittingly invite alan adult whos trapped inside game 26 years living room alans hope freedom finish game proves risky three find running giant rhinoceroses evil monkeys terrifying creatures'

In [57]:
df_sample.loc['Free Willy 2 - The Adventure Home'].at['overview']

'jesse becomes reunited willy three years whales jump freedom teenager tries rescue killer whale orcas oil spill'

In [67]:
df_sample.loc['Brainscan'].at['overview']

'lonely teenage horrormovie fan discovers mysterious computer game uses hypnosis customtailor game terrifying experience imaginable emerges hypnotic trance horrified find evidence brutal murder depicted game actually happened hes killer'

In [77]:
df_sample.loc['Word Wars'].at['overview']

'classic board game scrabble popular decades addition fanatics devote heart soul game expense everything else film profiles group enthusiasts converge scrabble convention word game almost bloodsport'

In [95]:
df_sample.loc['Indecent Proposal'].at['overview']

'robert redford stars billionaire john gage offers downonhisluck yuppie husband woody harrelson 1 million opportunity spend night mans wife demi moore'

In [96]:
df_sample.loc['Toy Story'].at['overview']

'led woody andys toys live happily room andys birthday brings buzz lightyear onto scene afraid losing place andys heart woody plots buzz circumstances separate buzz woody owner duo eventually learns put aside differences'

In [97]:
df_sample.loc['Condorman'].at['overview']

'comic artist writer woody performs simple courier operation friend harry works cia successfully fends hostile agents earns respect beautiful natalia requests assistance defection woody uses request leverage use cias resources bring comic book creation condorman life battle evil krokov'

In [98]:
df_sample.loc['Rebel Without a Cause'].at['overview']

'moving new town troublemaking teen jim stark supposed clean slate although new kid town brings problems searching stability stark forms bond disturbed classmate plato falls local girl judy however judy girlfriend neighborhood tough buzz buzz violently confronts jim challenges drag race new kids real troubles begin'

# # Recommendation based on movie title


In [122]:
df_title=movies[['title']]

In [123]:
df_title['title']=df_title['title'].astype(str)

C:\Users\abishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [124]:
df_title['title'].apply(lambda x: len(x.split(' '))).sum()

136495

In [127]:
df_title.shape

(45466, 1)

# To build recommendation based on movie title we need extract features fom Title column
So convert title columns as bag of words

In [125]:

count_title = CountVectorizer()
count_matrix_title = count.fit_transform(df_title['title'])

In [128]:
count_matrix_title.toarray().shape

(45466, 23097)

In [118]:
df_title.set_index('title', inplace = True)
df_title.head()

""
title
Toy Story
Jumanji
Grumpier Old Men
Waiting to Exhale
Father of the Bride Part II


In [119]:
indices = pd.Series(df_title.index)
indices[:5]

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

# generating the cosine similarity matrix

In [130]:

cosine_sim = cosine_similarity(count_matrix_title, count_matrix_title)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [132]:
# function that takes in movie title as input and returns the top 20 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 20 most similar movies
    top_20_indexes = list(score_series.iloc[1:21].index)
    
    # populating the list with the titles of the best 20 matching movies
    for i in top_20_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

As you can see the Recommended list all the title are similiar with either  'Toy' or 'Story'

In [133]:
recommendations('Toy Story')

['Toy Story 2',
 'Toy Story 3',
 'L.A. Story',
 'Toy Story of Terror!',
 'Toy Story That Time Forgot',
 'A Christmas Story 2',
 'Straight Story',
 'Hate Story',
 'Hate Story 2',
 'A Christmas Story',
 'Japanese Story',
 'Gangster Story',
 'Hollywood Story',
 "Oliver's Story",
 "A Soldier's Story",
 'Internet Story',
 'Malta Story',
 "Kid's Story",
 'Love Story',
 'Brave Story']

In [134]:
recommendations('Grumpier Old Men')

['Old Men: Robbers',
 'Grumpy Old Men',
 'Men...',
 'X-Men',
 'G-Men',
 'T-Men',
 'A Gathering of Old Men',
 'No Country for Old Men',
 'Seven Old Men and One Girl',
 'Only Old Men Are Going to Battle',
 'Men Do what Men Can',
 'Tin Men',
 'Old Dogs',
 'Soul Men',
 'Inside Men',
 'Old Partner',
 'Old Joy',
 'Old School',
 'Safe Men',
 'Old Hags']